## Az eredményeket mentsd a src/weekly modul-ba weekly_test_6.py néven

#### Használható modulok: pathlib, pandas, typing, str, statsmodels

In [ ]:
from pathlib import Path

import fastparquet
import statsmodels

In [ ]:
datalib = Path.cwd().parent.joinpath('data')

In [ ]:
pip install fastparquet

In [ ]:
pip install statsmodels

In [ ]:
"""
1., Olvasd be a data mappa sp500.parquet nevű fájlját egy DataFrame-be. A betöltéshez használt engine paramétere legyen fastparquet
"""

In [12]:
import pandas as pd

# Load data from 'sp500.parquet' using the 'fastparquet' engine with the full path
df_sp500 = pd.read_parquet('C:/Users/Atti60/Documents/GitHub/ECOPY_23241/data/sp500.parquet', engine='fastparquet')



In [ ]:
"""
2., Olvasd be az ff_factors.parquet fájlt egy DataFrame-be. A betöltéshez használt engine paramétere legyen fastparquet
"""

In [13]:
# Load data from 'ff_factors.parquet' using the 'fastparquet' engine
df_factors = pd.read_parquet('C:/Users/Atti60/Documents/GitHub/ECOPY_23241/data/ff_factors.parquet', engine='fastparquet')


In [ ]:
"""
3., Kapcsold össze a két DataFrame-t egy új DataFrame-be. Az összekapcsolás módja, hogy a hozam adatokra balról kapcsoljuk rá a factor adatokat a 'Date' elsődleges kulcs alapján.
"""

![Joined data](../resources/weekly6/joined_data.jpg)

In [14]:
merged_df = pd.merge(df_sp500, df_factors, on='Date', how='left')


In [ ]:
"""
4., Készíts egy új 'Excess Return' nevű oszlopot, ami a havi hozamok és a kockázat mentes hozam (RF) különbsége
"""

In [15]:
merged_df['Excess Return'] = merged_df['Monthly Returns'] - merged_df['RF']


In [ ]:
"""
5., Rendezd sorba dátum szerint az adatokat, majd generálj egy új oszlopot ('ex_ret_1'), amely minden ticker ('Symbol') esetén 1-el eltolja az Excess Return értékeit olyan módon, hogy minden sorban szerepeljen a következő időszaki Excess Return érték. 
"""

![new column](../resources/weekly6/ex_ret_1.jpg)

In [16]:
merged_df['ex_ret_1'] = merged_df.groupby('Symbol')['Excess Return'].shift(-1)


In [ ]:
"""
6., A meglévő adathalmazt írd felül olyan módon, hogy egyszer törlöd az össze olyan sort, amely az 'ex_ret_1' oszlopban hiányos, majd ezt követően, törlöd az összes olyan sort, ami a 'HML' oszlopban hiányos
"""

In [17]:
merged_df.dropna(subset=['ex_ret_1', 'HML'], inplace=True)


In [ ]:
"""
A meglévő adatokból válaszd ki a Amazon részvényhez tartozó sorokat (AMZN) és töröld a tickereket tartalmazó oszlopot

7., Készíts egy új LinearRegressionSM elnevezésű osztályt. Definiáld benne a __init__ nevű függvényt, amely bemenetként 2 DataDrame-t kap amelyeket ments le a left_hand_side és right_hand_side elnevezésű változókba. Az egyik DataFrame fogja tartalmazni a következő hónap többlet hozamait (left_hand_side), a másik a piaci hozamokat (Mkt-RF), az SMB és a HML értékeket (right_hand_side).

8., Egésztsd ki az osztályt egy fit metódussal, ami meghívja a statsmodels segítségével OLS alapon becsült modellt illeszt az osztály adattagjaira. A becslésből visszakapott modellt mentse le az osztály _model adattagjába. Figyelj oda, hogy a regresszió futtatása során konstans (alfa / béta_0) is szerepeljen a predictor változók között.

9., Egészítsd ki az osztályt egy get_params metódussal, ami visszaadja a becsült modellt béta paramétereinek értékeit. A visszakapott pandas Series típusú adatban az oszlop neve legyen 'Beta coefficients'

10., Egészítsd ki az osztályt egy get_pvalues metódussal, ami visszaadja a becsült modell paraméterekhez tartozó p értékeket. A visszakapott pandas Series típusú adatban az oszlop neve legyen: P-values for the corresponding coefficients

11., Egészítsd ki az osztályt egy get_wald_test_result metódussal, ami visszaadja a bemeneti restrikciós mátrix alapján számolt F és p értékeket. A visszatérési típus string legyen, a visszaadandó szöveg: F-value: fvalue, p-value: pvalue, ahol az fvalue és pvalue helyére 3 tizedesjegyre kerekítve adja meg a hozzájuk tartozó értékeket.

12., Egészítse ki az osztályt egy get_model_goodness_values metódussal, ami visszadja a módosított R-négyzet, az Akaike és a Bayes információs kritériumok értékét. A visszatérési típus string legyen, a visszaadandó szöveg: Adjusted R-squared: ars, Akaike IC: ak, Bayes IC: by, ahol ars, ak és by helyére 3 tizedesjegyre kerekítve adja meg a hozzájuk tartozó értékeket.

A létrehozott osztályt és a hozzátartozó metódusokat mentsd le a src.linear_regression mappába, a LinearRegressions.py fájlba.
"""

In [19]:
amazon_df = merged_df[merged_df['Symbol'] == 'AMZN'].copy()
amazon_df.drop('Symbol', axis=1, inplace=True)

import statsmodels.api as sm

class LinearRegressionSM:
    def __init__(self, left_hand_side, right_hand_side):
        self.left_hand_side = left_hand_side
        self.right_hand_side = right_hand_side
        self._model = None

    def fit(self):
        X = sm.add_constant(self.right_hand_side)
        model = sm.OLS(self.left_hand_side, X).fit()
        self._model = model

    def get_params(self):
        return pd.Series(self._model.params, name='Beta coefficients')

    def get_pvalues(self):
        return pd.Series(self._model.pvalues, name='P-values for the corresponding coefficients')

    def get_wald_test_result(self, restriction_matrix):
        wald_test = self._model.wald_test(restriction_matrix)
        fvalue = round(wald_test.statistic, 3)
        pvalue = round(wald_test.pvalue, 3)
        return f'F-value: {fvalue}, p-value: {pvalue}'

    def get_model_goodness_values(self):
        ars = round(self._model.rsquared_adj, 3)
        ak = round(self._model.aic, 3)
        by = round(self._model.bic, 3)
        return f'Adjusted R-squared: {ars}, Akaike IC: {ak}, Bayes IC: {by}'


ModuleNotFoundError: No module named 'statsmodels'